In [1]:
import numpy as np
import pandas as pd
import torch
import torch.nn.functional as F
import torchtext
import time
import random
import pandas as pd
import spacy

In [2]:
from google.colab import files
uploaded = files.upload()

Saving IMDB Dataset.csv to IMDB Dataset (1).csv


# Import Required Libraries & Data Loading

In [3]:
#importing the training data
import io
df=pd.read_csv(io.BytesIO(uploaded['IMDB Dataset.csv']))
print(df.shape)
df.head(10)

(50000, 2)


,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive
5,"Probably my all-time favorite movie, a story o...",positive
6,I sure would like to see a resurrection of a u...,positive
7,"This show was an amazing, fresh & innovative i...",negative
8,Encouraged by the positive comments about this...,negative
9,If you like original gut wrenching laughter yo...,positive


In [4]:
!pip install torchtext==0.9.0


# Data Preparation

In [5]:
"""
sentiment : 0 = negative, 1 = positive 
use the following to get the sentiment of a sentence :  
sentiment = 0 if sentiment is negative else 1


use np.where to get the sentiment of a sentence :
"""
df['sentiment'] = np.where(df['sentiment'] == 'positive', 1, 0)

In [6]:
df.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,1
1,A wonderful little production. <br /><br />The...,1
2,I thought this was a wonderful way to spend ti...,1
3,Basically there's a family where a little boy ...,0
4,"Petter Mattei's ""Love in the Time of Money"" is...",1


In [7]:
df.columns = ['TEXT_COLUMN_NAME', 'LABEL_COLUMN_NAME']

In [8]:
"""
Load the spacy model and load the English language model from https://spacy.io/usage/models
"""

!pip install spacy
!python -m spacy download en_core_web_sm

     |████████████████████████████████| 12.0 MB 5.0 MB/s 
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')


In [9]:
RANDOM_SEED = 150
torch.manual_seed(RANDOM_SEED)

VOCABULARY_SIZE = 20000
LEARNING_RATE = 0.005
BATCH_SIZE = 128
NUM_EPOCHS = 15
DEVICE ='cuda' if torch.cuda.is_available() else 'cpu'
EMBEDDING_DIM = 128
HIDDEN_DIM = 256
NUM_CLASSES = 2

Text & label Preparation

In [10]:
# Define feature processing

TEXT = torchtext.legacy.data.Field(tokenize='spacy', tokenizer_language='en_core_web_sm'
)



In [11]:
# Define Label processing
LABEL = torchtext.legacy.data.LabelField(dtype=torch.long)

In [12]:
"""
Define the fields for the data.
"""

df.to_csv('moviedata.csv', index = None)
df = pd.read_csv('moviedata.csv')
df.head()

,TEXT_COLUMN_NAME,LABEL_COLUMN_NAME
0,One of the other reviewers has mentioned that ...,1
1,A wonderful little production. <br /><br />The...,1
2,I thought this was a wonderful way to spend ti...,1
3,Basically there's a family where a little boy ...,0
4,"Petter Mattei's ""Love in the Time of Money"" is...",1


In [14]:
# process the dataset

fields = [('TEXT_COLUMN_NAME', TEXT), ('LABEL_COLUMN_NAME', LABEL)]

dataset = torchtext.legacy.data.TabularDataset(
                    path = "IMDB Dataset.csv", 
                    format = "csv", 
                    skip_header = True, 
                    fields =  fields,
)

# Data Split

In [15]:
# Split dataset into train and test set

train_data, test_data = dataset.split(split_ratio = [0.8, 0.2], random_state = random.seed(RANDOM_SEED))

print('Length of train data', len(train_data))
print('Length of test data', len(test_data))

Length of train data 40000
Length of test data 10000


In [16]:
train_data, val_data = train_data.split(split_ratio = [0.85, 0.15], random_state = random.seed(RANDOM_SEED))

print('Length of train data', len(train_data))
print('Length of valid data', len(val_data))

Length of train data 34000
Length of valid data 6000


# Data Observation after Tokenization


In [17]:
print(vars(train_data.examples[2000]))

{'TEXT_COLUMN_NAME': ['When', 'my', 'six', '-', 'year', '-', 'old', 'fell', 'asleep', 'at', 'the', 'theater', 'during', 'this', 'movie', ',', 'that', 'was', 'all', 'the', 'confirmation', 'I', 'needed', 'that', 'this', 'film', 'is', 'a', 'stinker', '.', 'It', 'was', 'boring', '.', 'The', 'scary', 'parts', 'were', 'boring', '.', 'The', 'maudlin', 'parts', 'were', 'boring', '.', 'Even', 'the', 'funny', 'parts', '--', 'with', 'two', 'minor', 'exceptions', '--', 'were', 'boring', '.', 'And', 'predictable', '.', 'And', 'did', 'I', 'mention', 'predictable', '?', 'Examples', ':', '1', ')', 'The', 'gruff', 'but', 'loving', 'Dad', 'indulging', 'his', 'brat', 'of', 'a', 'kid', '.', '2', ')', 'The', 'gruff', 'but', 'loving', 'Dad', 'buying', 'the', 'farm', '(', 'while', 'singing', 'a', 'song', '?', '!', ')', '.', '3', ')', 'The', 'brat', 'of', 'a', 'kid', 'deciding', 'to', 'cut', 'and', 'run', ',', 'but', ',', 'thanks', 'to', 'his', 'friends', ',', 'sticking', 'things', 'out', 'because', 'they', '

In [18]:
# Build Vocabulary

TEXT.build_vocab(train_data, max_size = VOCABULARY_SIZE)
LABEL.build_vocab(train_data)

print(f'vocabulary size: {len(TEXT.vocab)}')
print(f'Label Size: {len(LABEL.vocab)}')

vocabulary size: 20002
Label Size: 2


 2 extra value in vocabulary is because added (unknown) and (padding)

In [19]:
# Print the most common words: Use the most_common method of the TEXT vocabulary
most_common_words = TEXT.vocab.freqs.most_common(20)
print(most_common_words)

[('the', 391096), (',', 369258), ('.', 319400), ('a', 210381), ('and', 210039), ('of', 194441), ('to', 179866), ('is', 146338), ('in', 118386), ('I', 105565), ('it', 103531), ('that', 94022), ('"', 86342), ("'s", 82530), ('this', 81891), ('-', 70523), ('/><br', 69078), ('was', 67421), ('as', 57894), ('movie', 57845)]


In [20]:
# Token corresponding to first 10 Indices

print(TEXT.vocab.itos[:20]) #itos = Integer to string

['<unk>', '<pad>', 'the', ',', '.', 'a', 'and', 'of', 'to', 'is', 'in', 'I', 'it', 'that', '"', "'s", 'this', '-', '/><br', 'was']


# Data Preparation for Batch wise Implimentation

In [21]:
# Define Dataloader
train_loader, valid_loader, test_loader = torchtext.legacy.data.BucketIterator.splits(
        (train_data, val_data, test_data),
         batch_size=BATCH_SIZE,
         sort_within_batch=False,
         sort_key=lambda x: len(x.TEXT_COLUMN_NAME),
         device=DEVICE
    )

In [22]:
# Testing the iterators (note that the number of rows depends on the longest document in the respective batch):

print('Train')
for batch in train_loader:
    print(f'Text matrix size: {batch.TEXT_COLUMN_NAME.size()}')
    print(f'Target vector size: {batch.LABEL_COLUMN_NAME.size()}')
    break
    
print('\nValid:')
for batch in valid_loader:
    print(f'Text matrix size: {batch.TEXT_COLUMN_NAME.size()}')
    print(f'Target vector size: {batch.LABEL_COLUMN_NAME.size()}')
    break
    
print('\nTest:')
for batch in test_loader:
    print(f'Text matrix size: {batch.TEXT_COLUMN_NAME.size()}')
    print(f'Target vector size: {batch.LABEL_COLUMN_NAME.size()}')
    break

Train
Text matrix size: torch.Size([921, 128])
Target vector size: torch.Size([128])

Valid:
Text matrix size: torch.Size([58, 128])
Target vector size: torch.Size([128])

Test:
Text matrix size: torch.Size([51, 128])
Target vector size: torch.Size([128])


# Model Building

In [23]:
class RNN(torch.nn.Module):
    
    def __init__(self, input_dim, embedding_dim, hidden_dim, output_dim):
        super().__init__()

        self.embedding = torch.nn.Embedding(input_dim, embedding_dim)
        self.rnn = torch.nn.LSTM(embedding_dim,
                                 hidden_dim)     
        self.fc = torch.nn.Linear(hidden_dim, output_dim)
        

    def forward(self, text):
       
        embedded = self.embedding(text)
        
        output,(hidden,cell) = self.rnn(embedded)
        hidden.squeeze_(0)
        output = self.fc(hidden)
        return output

In [24]:
torch.manual_seed(RANDOM_SEED)
model = RNN(input_dim=len(TEXT.vocab),
            embedding_dim=EMBEDDING_DIM,
            hidden_dim=HIDDEN_DIM,
            output_dim=NUM_CLASSES # could use 1 for binary classification
)

model = model.to(DEVICE)
optimizer = torch.optim.Adam(model.parameters(),lr=0.005)

# Define Accuracy

In [25]:
def compute_accuracy(model, data_loader, device):

    with torch.no_grad():

        correct_pred, num_examples = 0, 0

        for i, (features, targets) in enumerate(data_loader):

            features = features.to(device)
            targets = targets.float().to(device)

            logits = model(features)
            _, predicted_labels = torch.max(logits, 1)

            num_examples += targets.size(0)
            correct_pred += (predicted_labels == targets).sum()
    return correct_pred.float()/num_examples * 100

# Model Run


In [26]:
start_time = time.time()

for epoch in range(NUM_EPOCHS):
    model.train()
    for batch_idx, batch_data in enumerate(train_loader):
        
        text = batch_data.TEXT_COLUMN_NAME.to(DEVICE)
        labels = batch_data.LABEL_COLUMN_NAME.to(DEVICE)

        ### FORWARD AND BACK PROP
        logits = model(text)
        loss = F.cross_entropy(logits, labels)
        optimizer.zero_grad()
        
        loss.backward()
        
        ### UPDATE MODEL PARAMETERS
        optimizer.step()
        
        ### LOGGING
        if not batch_idx % 50:
            print (f'Epoch: {epoch+1:03d}/{NUM_EPOCHS:03d} | '
                   f'Batch {batch_idx:03d}/{len(train_loader):03d} | '
                   f'Loss: {loss:.4f}')

    with torch.set_grad_enabled(False):
        print(f'training accuracy: '
              f'{compute_accuracy(model, train_loader, DEVICE):.2f}%'
              f'\nvalid accuracy: '
              f'{compute_accuracy(model, valid_loader, DEVICE):.2f}%')
        
    print(f'Time elapsed: {(time.time() - start_time)/60:.2f} min')
    
print(f'Total Training Time: {(time.time() - start_time)/60:.2f} min')
print(f'Test accuracy: {compute_accuracy(model, test_loader, DEVICE):.2f}%')

Epoch: 001/015 | Batch 000/266 | Loss: 0.6933
Epoch: 001/015 | Batch 050/266 | Loss: 0.6948
Epoch: 001/015 | Batch 100/266 | Loss: 0.6941
Epoch: 001/015 | Batch 150/266 | Loss: 0.6926
Epoch: 001/015 | Batch 200/266 | Loss: 0.6935
Epoch: 001/015 | Batch 250/266 | Loss: 0.6933
training accuracy: 50.04%
valid accuracy: 49.75%
Time elapsed: 2.28 min
Epoch: 002/015 | Batch 000/266 | Loss: 0.6932
Epoch: 002/015 | Batch 050/266 | Loss: 0.6902
Epoch: 002/015 | Batch 100/266 | Loss: 0.6956
Epoch: 002/015 | Batch 150/266 | Loss: 0.6911
Epoch: 002/015 | Batch 200/266 | Loss: 0.7092
Epoch: 002/015 | Batch 250/266 | Loss: 0.7018
training accuracy: 49.99%
valid accuracy: 49.62%
Time elapsed: 4.57 min
Epoch: 003/015 | Batch 000/266 | Loss: 0.6911
Epoch: 003/015 | Batch 050/266 | Loss: 0.6924
Epoch: 003/015 | Batch 100/266 | Loss: 0.6918
Epoch: 003/015 | Batch 150/266 | Loss: 0.6916
Epoch: 003/015 | Batch 200/266 | Loss: 0.6967
Epoch: 003/015 | Batch 250/266 | Loss: 0.6921
training accuracy: 50.18%
va

# Model Testing

In [27]:
import spacy


nlp = spacy.blank("en")

def predict_sentiment(model, sentence):

    model.eval()
    tokenized = [tok.text for tok in nlp.tokenizer(sentence)]
    indexed = [TEXT.vocab.stoi[t] for t in tokenized]
    length = [len(indexed)]
    tensor = torch.LongTensor(indexed).to(DEVICE)
    tensor = tensor.unsqueeze(1)
    length_tensor = torch.LongTensor(length)
    prediction = torch.nn.functional.softmax(model(tensor), dim=1)
    return prediction[0][1].item()

print('Probability positive:')
predict_sentiment(model, "This is such an awesome movie, I really love it!")

Probability positive:


0.998146653175354

In [28]:
print('Probability positive:')
predict_sentiment(model, "I really hate this movie. It is really bad and sucks!")

Probability positive:


0.007857277989387512